In [1]:
import numpy as np
import os
from astroquery.jplhorizons import Horizons

In [ ]:
# Paths to directories containing data
home_path = os.path.expanduser('~/')
path_67p = home_path+'Documents/year1/shape_modelling/67p/'

In [ ]:
# Constants
km2au = 1.496e8
c = 3e5

In [ ]:
### Function to create a file in the 'Mikko' format required for shape modelling 

def mikkoWrite(filename,day_list,int_list,sunx,suny,sunz,earthx,earthy,earthz, min_dist):
# Code to write to lightcurve file in Mikko format
# Additional argument to limit files to points beyond min_dist from the sun

    counter = 0
    fout = open(filename, 'w+')
    sameday = []
    for i, day in enumerate(day_list[:-1]):
        if (np.sqrt(sunx[i]**2+suny[i]**2+sunz[i]**2) >= min_dist):
            newday = True
            sameday.append(day)
            if abs(day-day_list[i+1])<0.5:
                newday = False
            if newday==False:
                if day not in sameday:
                    sameday.append(day)
            else:
                fout.write('%i 1\n'%(len(sameday)))
                for time in sameday:
                    loc = day_list.index(time)
                    fout.write('%.6f %.8f %.10f %.8f %.8f %.8f %.8f %.8f\n'%(time+2400000.5,int_list[loc],sunx[loc], suny[loc], sunz[loc], earthx[loc], earthy[loc], earthz[loc]))
                counter = counter+1
                sameday = []
    fout.write('1 1\n')
    fout.write('%.6f %.8f %.10f %.8f %.8f %.8f %.8f %.8f\n'%(day_list[-1]+2400000.5,int_list[-1],sunx[-1], suny[-1], sunz[-1], earthx[-1], earthy[-1], earthz[-1]))
    counter = counter+1
    fout.close()
    return counter

In [ ]:
# Read LSST sim output

lsst_file_name = path_67p+'LSST_out_LC_67P.txt'
lsst_file = open(lsst_file_name, 'r')

lsst_mjd =[]; lsst_delta=[]
lsst_ra = []; lsst_dec = []
lsst_sunx = []; lsst_suny = []; lsst_sunz = []
lsst_phase_ang = [];
lines_lsst = lsst_file.readlines()
for line in lines_lsst[25:]:
    dat = line.split()
    lsst_mjd.append(float(dat[2]))
    lsst_delta.append(float(dat[3]))
    lsst_ra.append(float(dat[5]))
    lsst_dec.append(float(dat[7]))
    lsst_sunx.append(float(dat[9]))
    lsst_suny.append(float(dat[10]))
    lsst_sunz.append(float(dat[11]))
    lsst_phase_ang.append(float(dat[21]))

lsst_delta = np.array(lsst_delta)/km2au
lsst_sunx = np.array(lsst_sunx)/km2au
lsst_suny = np.array(lsst_suny)/km2au
lsst_sunz = np.array(lsst_sunz)/km2au

In [ ]:
# Check it got everything
print(len(lsst_mjd))

In [ ]:
mjd = np.array(lsst_mjd)
sun_x = lsst_sunx
sun_y = lsst_suny
sun_z = lsst_sunz

- - - - - - - - - - - - -

In [ ]:
# LSST output only has obj-Sun geom: also need Earth-obj
# Also want list of solar phase angles, to compute abs mag from phase function
# ^Phase angles in LSST output

# Splitting this into 2 because Horizons wouldn't let me do all 678 epochs at once :(
obj = Horizons(id='90000694', location='I11',epochs=mjd[:500])

In [ ]:
earth_x = []; earth_y=[]; earth_z=[];
ecl_lon = []; ecl_lat = []
vec = obj.ephemerides()#refplane='earth')  # LSST vectoral output in equatorial ref frame!!!

for i in range(len(vec)):
    ecl_lon.append(float(vec['EclLon'][i]))
    ecl_lat.append(float(vec['EclLat'][i]))
#    earth_x.append(float(vec['x'][i]))
#    earth_y.append(float(vec['y'][i]))
#    earth_z.append(float(vec['z'][i]))

In [ ]:
# Check output looks sensible
vec

In [ ]:
# Now get the remaining 655-500 vectors
obj = Horizons(id='90000694', location='I11', epochs=mjd[500:])

In [ ]:
vec = obj.ephemerides()#refplane='earth')  
for i in range(len(vec)):
    ecl_lon.append(float(vec['EclLon'][i]))
    ecl_lat.append(float(vec['EclLat'][i]))
#for i in range(len(vec['x'])):
    #earth_x.append(float(vec['x'][i]))
    #earth_y.append(float(vec['y'][i]))
    #earth_z.append(float(vec['z'][i]))

In [ ]:
for i in range(678):
    print(f'{ecl_lon[i]:.4f}  {ecl_lat[i]:.4f}')

In [ ]:
# Check this didn't miss anything or double anything
len(earth_x)

In [ ]:
# Light time correct the MJDs:
delta = lsst_delta # in km
#for i in range(len(earth_x)):
#    delta.append(np.sqrt(earth_x[i]**2+earth_y[i]**2+earth_z[i]**2))
#delta=np.array(delta)
mjd_ltc = np.zeros_like(mjd)

mjd_ltc = mjd-(((delta*km2au)/c)/(3600*24))
mjd_ltc = np.ndarray.tolist(mjd_ltc)

In [ ]:
for i,time in enumerate(mjd_ltc):
    print(time, mjd[i])

In [ ]:
sun_xyz = np.array([sun_x, sun_y, sun_z])
print(sun_xyz[:,0])
earth_xyz = np.array([earth_x, earth_y, earth_z])
print(earth_xyz[:,0])

In [ ]:
namestr = path_67p+'test.txt'
mikkoWrite(namestr,mjd_ltc,np.ones_like(mjd),sun_x,sun_y,sun_z,earth_x,earth_y,earth_z,0.0)
# Go and type the number output rom this cell into the top of the file you've just created
# This is the number of lightcurves in total

In [ ]:
# Checking the phase angles: should make this into a function but too lazy
alpha_list_calc = []
for i in range(len(earth_xyz[1])):
    alpha_list_calc.append((360/(2.*np.pi))*np.arccos(np.dot(sun_xyz[:,i],earth_xyz[:,i])/(np.linalg.norm(sun_xyz[:,i])*np.linalg.norm(earth_xyz[:,i]))))

In [ ]:
# This is just to grab ephemeris data to check phase angles
# Will also use this to grab rh values for completion's sake
obj = Horizons(id='90000696', location='500', epochs=mjd[:500])
ephem_table = obj.ephemerides()

In [ ]:
ephem_table

In [ ]:
alpha_list = []; rh_list = []
for i,row in enumerate(ephem_table['alpha_true']):
    alpha_list.append(float(row))
    rh_list.append(float(ephem_table['r'][i]))

In [ ]:
obj = Horizons(id='90000696', location='geocentric', epochs=mjd[500:])
ephem_table = obj.ephemerides()

In [ ]:
# Append the rest of the values to the lists
for i,row in enumerate(ephem_table['alpha_true']):
    alpha_list.append(float(row))
    rh_list.append(float(ephem_table['r'][i]))

In [ ]:
for i in range(len(alpha_list)):
    print(alpha_list[i], lsst_phase_ang[i])

In [ ]:
####-----Go and use mag_from_phase_func.py to get shifted absolute magnitudes-----#####

In [ ]:
# Read in shifted magnitudes:
shifted_mags_file = path_67p+'67P_mag_list_ALL.txt'
shifted_mags_list = []
magf=open(shifted_mags_file, 'r')
lines = magf.readlines()
for line in lines:
    data = line.split()
    shifted_mags_list.append(float(data[0]))
#shifted_mags_list

In [ ]:
generate_intensities = []
for i in range(len(shifted_mags_list)):
    intensity=10.**(-0.4*(shifted_mags_list[i]-corr_fac))
    generate_intensities.append(intensity)

In [ ]:
# Final step: read in intensities as calculated from shifts from Matlab script
# Obvs if you're using this script to create Mikko file in the first instance, you won't have these
intensities_file = path_67p+'67P_int_list_ALL.txt'
intensities_list = []
intf = open(intensities_file, 'r')
lines = intf.readlines()
for line in lines:
    data = line.split()
    intensities_list.append(float(data[0]))
intensities_list

In [ ]:
# Create Mikko file with real intensities (not dummy)
print(len(generate_intensities))

namestr_fin = path_67p+'67P_LC_NU.dat'
mikkoWrite(namestr_fin,mjd_ltc,generate_intensities,sun_x,sun_y,sun_z,earth_x,earth_y,earth_z,0.0)
# Go and type the number output rom this cell into the top of the file you've just created
# This is the number of lightcurves in total

In [ ]:
# Also making standard lightcurve output with dates, magnitudes and 0.01 mag. uncertainty

corr_fac = 16    # from mag_from_phase_func py script

fout = open(path_67p+'67p_20230109_I11_R.dat', 'w+')
for i,day in enumerate(mjd_ltc):
    #magnitude = corr_fac -2.5*np.log10(intensities_list[i])
    magnitude = shifted_mags_list[i]
    fout.write('%.6f %.4f 0.01 %.8f %.8f %.8f %.8f %.8f %.8f %.8f %.8f\n'%(day+2400000.5, magnitude, sun_x[i], sun_y[i], sun_z[i], rh_list[i], earth_x[i], earth_y[i], earth_z[i], delta[i], ecl_lon[i], ecl_lat[i]), overwrite=False)
fout.close()

In [ ]:
generate_intensities